---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Datasets

En la práctica de la clase de hoy usaremos un **conjunto de propiedades** que tienen a la venta la inmobiliaria Properati: https://www.properati.com.ar/. Lo interesante es que a los datos clásicos de la propiedad (valor, superficie, barrio, tipo de propiedad), le agrega la posición geoespacial mediante su latitud y longitud.

Por otra parte, vamos a considerar a las **estaciones de subte**, donde tambien figuran el nombre y la línea a la que pertenece, y además, obviamente, sus datos geoposicionales.

## Ejercicio

Comenzamos leyendo los dos datasets en un *dataframe*, y lo transformamos en un *GeoDataFrame*. Es el tipo de datos que requiere GeoPandas para realizar operaciones con datos geoespaciales.

Luego vamos a *calcular la distancia* de cada propiedad al obelisco de Buenos Aires, y se registrará en una nueva columna. Haremos un cálculo para ver si existe alguna relacion entre esta distancia y el precio de la propiedad.

Finalmente *trazaremos una línea geométrica* con todas las estaciones de subte de una línea, y la graficaremos sobre la ciudad.

Alla vamos!

---

Importamos las bibliotecas que vamos a necesitar:

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import shapely # genera las figuras geometricas
import descartes # relaciona shapely con matplotlib
import pyproj # proyecciones. Transformar coordenadas

### Parte 1 - Archivo de propiedades

Vamos a leer los datos del archivo /M1/CLASE_07/Data/properati_caba.csv en un `DataFrame` de pandas con el método `read_csv`. Lo llamaremos df_prop. 

*Nota:* los datos vienen separados por tabs. Se debe usar el parámetro <code>sep='\t'</code>

Ayuda:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

- 1.a. Tomamos una muestra de 5 elementos

In [ ]:
# 1.a


- 1.b. Cuántas propiedades son? Cuántas columnas tiene?

In [ ]:
# 1.b


- 1.c. Cuáles son las columnas? De qué tipo de datos son?

In [ ]:
# 1.c


- 1.d. ¿Cuantas propiedades tenemos por barrio? El barrio figura en la columna *place_name*.

Ayuda:

    - Libro McKinney - Python for Data Analysis 2ed.pdf - capitulo 10 Data Aggregation and Group Operations

    - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html?highlight=value_counts#pandas.Series.value_counts

In [ ]:
# 1.d.


- 1.e. Las columnas *lat* y *lon* identifican la *latitud* y la *longitud* de la posición geoespacial de las propiedades. Pero necesitamos que la posición se exprese mediante la forma geométrica Punto, para poder trabajarlo desde GeoPandas.

Es decir que debemos transformar el DataFrame en un GeoDataFrame. Lo llamaremos *geo_prop*.

Pero para generar un GeoDataFrame, necesitamos agregar a las columnas del DataFrame una nueva columna que contenga una forma geométrica. Es este caso, un *punto*, el cual se genera a partir de la *latitud* y la *longitud*. 

Es recomendable que la nueva columna se llame *geometry*.

Con el método `gpd.points_from_xy` lo realizamos.

*Observar* que para definir el tipo *punto*, primero se ingresa la **longitud** y luego la **latitud**.

Ayuda:
https://geopandas.org/reference/geopandas.points_from_xy.html

In [ ]:
# 1.e


Verificamos la nueva columna mirando las primeras filas

In [ ]:
# 1.e
geo_prop.loc[:4, ['geometry', 'lat', 'lon','property_type','place_name']]

---

### Parte 2 - Archivo de estaciones de subte

Vamos a leer los datos del archivo /M1/CLASE_07/Data/estaciones-de-subte.csv en un `DataFrame` de pandas con el método `read_csv`. Lo llamaremos df_subte.

*Nota:* los datos vienen separados por comas. Se debe usar el parámetro <code>sep=','</code>

- 2.a. Consultamos los primeros 10 registros

In [ ]:
# 2.a.


- 2.b. Cuántas estaciones son? Cuántas columnas tiene?

In [ ]:
# 2.b.


- 2.c. Cuáles son las columnas? De qué tipo de datos son?

In [ ]:
# 2.c.


- 2.d. ¿Cuantas estaciones tenemos por línea?

Ayuda: es similar al punto 1.d

In [ ]:
# 2.d.


- 2.e. Las columnas *lat* y *long* identifican la *latitud* y la *longitud* de la posición geoespacial de las propiedades. Pero necesitamos que la posición se exprese mediante la forma geométrica Punto, para poder trabajarlo desde GeoPandas.

Ayuda: es similar al punto 1.e

In [ ]:
# 2.e.


Verificamos la nueva columna mirando las primeras filas

In [ ]:
geo_subte.head(5)

---

### Parte 3 - Cálculo de la distancia

Vamos a calcular la **distancia de cada propiedad al obelisco de Buenos Aires**, y la registramos en una nueva columna.

Primero necesitamos representar el lugar geográfico del Obelisco de Buenos Aires. Lo hacemos mediante la forma geométrica *Punto*, y las cooordenadas del lugar. 

In [ ]:
from shapely.geometry import Point

punto_obelisco_p = Point(-58.381555,-34.605425)

In [ ]:
type(punto_obelisco_p)

Pero el método que calcula la distancia entre dos puntos, <code>geopy.distance.geodesic</code>, necesita representar el lugar geográfico del Obelisco de Buenos Aires mediante una *tupla*.

In [ ]:
punto_obelisco = (-58.381555,-34.605425)

In [ ]:
type(punto_obelisco)

Ahora podemos agregar la nueva columna con la distancia entre el punto del obelisco y el punto de cada propiedad (en metros). La llamaremos *distancia_obelisco*. 

Se debe generar **una iteración** sobre el indice de geo_prop:

Ayuda:
<code>
for i in geo_prop.index:
    geo_prop.loc[i, 'distancia_obelisco'] = calculo distancia entre obelisco y cada propiedad.</code>
    
El cálculo se realiza con el método <code>geopy.distance.geodesic</code>, el cual se explica en la notebook 3_geopandas_operaciones.

Ademas con la sentencia <code>geo_prop.loc[i,'geometry'].x</code> obtenemos la coordenada longitud de la distancia.

Debe terminar con <code>.meters</code> para indicar la distancia en metros

Ver https://geopy.readthedocs.io/en/stable/#module-geopy.distance

In [ ]:
# 3.a
import geopy.distance

for i in geo_prop.index:
    geo_prop.loc[i, 'distancia_obelisco'] = geopy.distance.geodesic(punto_obelisco,
                  (geo_prop.loc[i,'geometry'].x
                  ,geo_prop.loc[i,'geometry'].y)).meters

Verificamos la nueva columna mirando las primeras filas

In [ ]:
geo_prop.loc[:4, ['distancia_obelisco','geometry', 'lat', 'lon','property_type','place_name']]

Vamos a calcular una relación entre la distancia al obelisco y los precios de cada propiedad. 

Calculemos el promedio por barrio de la distancia y de los precios.Y luego hacemos un ranking de los promedios, y vemos si los primeros 10 rankeados de cada promedio son de los mismos barrios.

Para todos los calculos necesitamos usar el método **groupby**.

Primero se debe crear un nuevo GeoDataFrame con tantas filas como barrios tenemos.

In [ ]:
geo_prop_group = geo_prop.groupby(['place_name']).count()

In [ ]:
type(geo_prop_group)

In [ ]:
geo_prop_group

Luego debemos agregar dos columnas con los promedios (distancia y precio) por barrio.

Ayuda:
<code>
geo_prop_group['nueva columna'] = geo_prop.groupby(['xxxxxxx'])[["yyyy"]].mean()
</code>

In [ ]:
# 3.b. Promedio precios por barrio


In [ ]:
# 3.b.Promedio distancia por barrio


Y finalmente, agregar otras dos columnas con el ranking según el promedio (distancia y precio) por barrio.

Ayuda:
<code>
geo_prop_group["nueva columna rank"] = geo_prop_group.xxxxxxxx.rank(ascending=True)
</code>

In [ ]:
# 3.b.Ranking distancia por barrio


In [ ]:
# 3.b.Ranking precio por barrio


Verificamos las nuevas columnas

In [ ]:
geo_prop_group.loc[:, ['precio','distancia','rank_distancia','rank_precio']]

Finalmente, observar si los primeros 10 rankeados de cada promedio son de los mismos barrios.

Primero hacerlo sobre el ranking de distancia, y luego sobre el ranking de precios.

Ayuda: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.sort_values.html

In [ ]:
# 10 primeros rankeados por distancia

In [ ]:
# 10 primeros rankeados por precio

### Parte 4 - Línea que una a las estaciones de subte

Vamos a hacer una línea geométrica con las estaciones de subte de la línea H y graficarla sobre la ciudad.

<div>
    <div class = "mapa">
        <img src='img/M1_Clase_07_1_desafio.jpg' alt="Elementos geométricos" width=80% height=90%>
    </div>
</div>

In [ ]:
geo_subte.head(4)

Primero seleccionemos solo las estaciones de la linea H, y guardamos la selección en la variable geo_subte_h.

In [ ]:
geo_subte_h

Como muestra el dibujo, la línea corre de Norte a Sur (o Sur a Norte). Como las estaciones deben estar alineadas para que salga bien la línea, podemos usar la columna lat para ordenarlas; lat es la latitud de cada estación.

Ayuda: considerar sort_values con by
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

In [ ]:
geo_subte_h

Para simplificar la codificación, solo nos quedaremos con los datos geográficos de cada estación.

Generamos una *GeoSeries* con la columna geometry de geo_subte_h. Y la llamamos geo_subte_h_geometry.

Ahora creamos las líneas que unen a las estaciones de subte. 

Cada línea se forma con **dos puntos**. Por lo tanto, tenemos que tomar el primer y segundo punto de la Geoserie, y aplicarles el método <code>LineString</code>. Luego seguimos con el segundo y el tercero, y así siguiendo.

*Primero creamos una lista donde guardaremos cada linea.* La llamamos linea_h. Debe tener una longitud igual a la cantidad de estaciones menos 1, es decir longitud 11.

In [ ]:
longitud_linea = len(geo_subte_h_geometry)-1
linea_h = list(range(longitud_linea))

Ahora podemos crear las líneas y guardarlas en la lista.

Se debe generar **una iteración** para recorrer la GeoSerie *geo_subte_h_geometry*.

Ayuda:
<code>
for i in range(longitud_linea): 
    linea_h[i] = creo la linea entre el elemento i y el elemento i+1
</code> 
    
El cálculo se realiza con el método <code>LineString</code> de la libreria shapely, el cual se explica en la notebook 3_geopandas_operaciones.

In [ ]:
from shapely.geometry import LineString

for i in range(longitud_linea): 
    linea_h[i] = 

---

Ahora vamos a graficar las líneas sobre un mapa con los barrios de Buenos Aires.

Vamos a leer los datos del archivo /M1/CLASE_07/Data/barrios.csv en un `DataFrame` de pandas con el método `read_csv`. Lo llamaremos df_barrios.

---

Y luego generamos un GeoDataFrame de los barrios a partir del Dataframe. En este caso, la columna WKT ya contiene una forma geométrica **polígono** que representa el contorno de cada barrio. Pero en un formato WKT, que no permite generar un GeoDataFrame. Por lo tanto, lo tenemos que convertir a un formato *geometry*.

In [ ]:
import shapely.wkt

df_barrios["WKT"] = df_barrios["WKT"].apply(shapely.wkt.loads) 
geo_barrios = gpd.GeoDataFrame(df_barrios, geometry='WKT')

Por otra parte, necesitamos para graficar las líneas generar una GeoSerie donde cada elemento se compone de dos puntos y la línea entre ellos.

In [ ]:
linea_h_geo = list(range(longitud_linea))

for i in range(longitud_linea): 
    linea_h_geo[i] = gpd.GeoSeries([geo_subte_h_geometry.iloc[i], geo_subte_h_geometry.iloc[i+1], linea_h[i]])

Con el método plot generamos el gráfico:

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
geo_barrios.plot(ax=ax, color='white', edgecolor='black')

for i in range(len(geo_subte_h_geometry)-2): 
    linea_h_geo[i].plot(ax=ax, color='red')

plt.show();